# Quantum algorithm on CI graphs

## Import package

In [1]:
import sys
import numpy as np
import math
import random
import turtle
import qiskit
from qiskit import QuantumCircuit, execute, Aer
from qiskit import IBMQ
from qiskit.aqua.algorithms import Grover
from qiskit.aqua.components.oracles import TruthTableOracle
from qiskit.providers.ibmq import least_busy

C:\Users\UCER\anaconda3\envs\IBMQ\lib\site-packages\qiskit\aqua\__init__.py:86: DeprecationWarning: The package qiskit.aqua is deprecated. It was moved/refactored to qiskit-terra For more information see <https://github.com/Qiskit/qiskit-aqua/blob/main/README.md#migration-guide>
  warn_package('aqua', 'qiskit-terra')


## Parameters

In [2]:
N = 12
nearest_connect = 4 #0 < lower < upper <= N/2
farthest_connect = 6
prob = 0.3
state = 'remote' # remote or local

## IBMQ config

In [3]:
#IBMQ.enable_account('e00b757253f820dd6dc1f1e6f173651daa82fc45f76f37510aa8d9032c990536f2fdda1fe3eaeeea836d1b360136ca0c38952ed9e5d36011207ab9fa47cc4082')

## Set backend(local simulator or remote IBMQ)

In [4]:
if(state != 'remote'):
    # simulator
    backend = Aer.get_backend('qasm_simulator')
else:
    provider = IBMQ.get_provider(hub='ibm-q')
    #backend = provider.get_backend('ibmq_qasm_simulator')
    backend = provider.get_backend('ibmq_bogota')

## Check input functions

In [5]:
def check_input(N, nearest_connect, farthest_connect):
    if(nearest_connect <= 0 or farthest_connect <= 0 or N <= 0):
        print('error: nearest_connect should bigger than 0')
        exit()
    elif(nearest_connect > farthest_connect):
        print('error: nearest_connect should not bigger than farthest_connect')
        exit()
    elif(farthest_connect > N/2):
        print('error: farthest_connect should not exceed half of total node')
        exit()
    else:
        return

## Create Adjacency Matrix and Input Boolean array

In [6]:
def create_matrix(N, nearest_connect, farthest_connect):
    array = np.zeros((N,N))
    for i in range (N):
        for j in range(nearest_connect, farthest_connect+1):
            if(i+j>N-1):
                array[i][i+j-N]=1
            else:
                array[i][i+j] = 1
    for i in range (N):
        for j in range(nearest_connect, farthest_connect+1):
            if(i-j<0):
                array[i][i-j+N]=1
            else:
                array[i][i-j] = 1
    return array

def decision(probability):
    return random.random() < probability

## Plotting utility: Turtle

In [7]:
def plot(a,f):
    def draw(i,j,pos):
        turtle.penup()
        turtle.setposition(pos[j][0],pos[j][1])
        dir_vec = [pos[i][0]-pos[j][0],pos[i][1]-pos[j][1]]
        dir = math.atan(dir_vec[1]/dir_vec[0])
        #fixup
        if dir_vec[0]<0 :
            dir+=math.pi
        
        #radient to degree
        dir = dir*180/math.pi

        length = math.sqrt((dir_vec[0])*(dir_vec[0])+(dir_vec[1])*(dir_vec[1]))

        turtle.setheading(dir)
        turtle.pendown()
        turtle.forward(length)

    size = len(f)
    #remember cordinate
    table = np.zeros((size,2))
    screen = turtle.Screen()
    turtle.speed(-3)
    turtle.penup()
    turtle.setposition(100,0)
    turtle.pendown()
    turtle.setheading(90)
    turtle.hideturtle()
    degree=0
    index=0
    arc = 360/size
    #draw vertices
    while degree<360:

        if f[index]==0:
            turtle.color('blue')
        else:
            turtle.color('red')
        turtle.dot(10)
        turtle.color('black')
        turtle.write(str(index),font=("Verdana", 15, "normal"))
        table[index] = turtle.pos()
        turtle.penup()
        turtle.circle(100,extent=arc)
        turtle.pendown()
        degree+=arc
        index +=1
        
    #draw edges
    for i in range(size):
        for j in range(i):
            if a[i][j]==1:
                pass
                draw(i,j,table)

    print('finish plotting')
    turtle.mainloop()

## Classical functions

In [8]:
def find_min_one_idx(low_idx,high_idx,f):
    min_one_idx = -1
    for j in range(low_idx,high_idx+1):
        if(f[j]):
            min_one_idx = j
            break
    return min_one_idx

def find_max_one_idx(low_idx,high_idx,f):
    max_one_idx = -1
    for j in range(high_idx,low_idx-1,-1):
        if(f[j]):
            max_one_idx = j
            break
    return max_one_idx

def is_any_one(low_idx,high_idx,f):
    is_any_one_var = 0
    for k in range(low_idx,high_idx+1):
        if(k >= N):
            k = k - N
        if(f[k]):
            is_any_one_var = 1
            break
    return is_any_one_var

## Functions using quantum algorithm

In [9]:
def ndarray2str(ndarray):
    string = ''
    for i in ndarray:
        string = string + str(int(i))
    return string
    
def is_any_one_quantum(low_idx,high_idx,f):
    
    doubled_f = np.concatenate((f,f))
    output_str = ndarray2str(doubled_f[low_idx:high_idx+1])
    original_output_str_len = len(output_str)
    
    if original_output_str_len == 1:
        return doubled_f[low_idx]
    
    power = math.ceil(math.log(len(output_str),2))
    if pow(2,power) > len(output_str):
        output_str = output_str + '0'*(pow(2,power)-len(output_str))
    
    oracle = TruthTableOracle(output_str)
    grover = Grover(oracle)
    
    result = grover.run(backend, shots=100)
    counts = result['measurement']
    print(counts)
    
    max_counts = 0
    measurement_outcome = None

    for i in counts:
        if counts[i] > max_counts:
            max_counts = counts[i]
            measurement_outcome = int(i,2)
    print(measurement_outcome)
    if measurement_outcome > original_output_str_len - 1:
        return 0
    else:
        measured_idx = measurement_outcome + low_idx
        return int(doubled_f[measured_idx])

## main function

### Create graph and boolean input array

In [10]:
check_input(N, nearest_connect, farthest_connect)
a = create_matrix(N, nearest_connect, farthest_connect)

f = np.zeros(N)

for i in range(N):
    f[i] = decision(prob)
print("f:")
print(f)
print(f[0],f[3])

f:
[1. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 1.]


### Plot graph

In [11]:
#plot(a,f)

### Divided into chunks

In [12]:
if(farthest_connect != nearest_connect):
    chunk_size = farthest_connect - nearest_connect
    is_collision_in_chunk_arr = np.zeros(math.ceil(N/chunk_size))
    is_collision_in_chunk_arr_quantum = np.zeros(math.ceil(N/chunk_size))
    print("size::::",math.ceil(N/chunk_size))
    for i in range(math.ceil(N/chunk_size)):
        min_idx_in_chunk = chunk_size * i
        max_idx_in_chunk = min(chunk_size * (i+1) - 1,N-1)
        min_one_idx_in_chunk = find_min_one_idx(min_idx_in_chunk,max_idx_in_chunk,f)
        max_one_idx_in_chunk = find_max_one_idx(min_idx_in_chunk,max_idx_in_chunk,f)
       
        if(min_one_idx_in_chunk==-1):
            continue

        connect_low_idx = min_one_idx_in_chunk + nearest_connect
        connect_high_idx = max_one_idx_in_chunk + farthest_connect
        
        is_collision_in_chunk_arr[i] = is_any_one(connect_low_idx,connect_high_idx,f)
        is_collision_in_chunk_arr_quantum[i] = is_any_one_quantum(connect_low_idx,connect_high_idx,f)

    is_collision_var = is_any_one(0,len(is_collision_in_chunk_arr)-1,is_collision_in_chunk_arr)
    is_collision_var_quantum = is_any_one_quantum(0,len(is_collision_in_chunk_arr_quantum)-1,is_collision_in_chunk_arr_quantum)
else:
    is_collision_in_chunk_arr = np.zeros(N)
    for i in range(N):
        connect_low_idx = i + nearest_connect
        connect_high_idx = i + farthest_connect
        if(f[i]):
            is_collision_in_chunk_arr[i] = is_any_one(connect_low_idx, connect_high_idx,f)

    is_collision_var = is_any_one(0,len(is_collision_in_chunk_arr)-1,is_collision_in_chunk_arr)
    is_collision_var_quantum = is_any_one_quantum(0,len(is_collision_in_chunk_arr)-1,is_collision_in_chunk_arr)

size:::: 8


C:\Users\UCER\anaconda3\envs\IBMQ\lib\site-packages\qiskit\aqua\algorithms\amplitude_amplifiers\grover.py:215: DeprecationWarning: The package qiskit.aqua.algorithms.amplitude_amplifiers is deprecated. It was moved/refactored to qiskit.algorithms.amplitude_amplifiers (pip install qiskit-terra). For more information see <https://github.com/Qiskit/qiskit-aqua/blob/main/README.md#migration-guide>
  warn_package('aqua.algorithms.amplitude_amplifiers',
C:\Users\UCER\anaconda3\envs\IBMQ\lib\site-packages\qiskit\aqua\quantum_instance.py:135: DeprecationWarning: The class qiskit.aqua.QuantumInstance is deprecated. It was moved/refactored to qiskit.utils.QuantumInstance (pip install qiskit-terra). For more information see <https://github.com/Qiskit/qiskit-aqua/blob/main/README.md#migration-guide>
  warn_class('aqua.QuantumInstance',
C:\Users\UCER\anaconda3\envs\IBMQ\lib\site-packages\qiskit\utils\deprecation.py:62: DeprecationWarning: Using a qobj for run() is deprecated as of qiskit-aer 0.9.0 

{'01': 24, '11': 27, '00': 20, '10': 29}
2
{'10': 27, '00': 20, '01': 30, '11': 23}
1
{'01': 30, '10': 26, '00': 19, '11': 25}
1
{'10': 100}
2
{'10': 25, '11': 26, '01': 27, '00': 22}
1
{'01': 100}
1
{'001': 53, '110': 47}
1


### Print result

In [13]:
print('quantum result:')
print(is_collision_var_quantum)
print('classical result:')
print(is_collision_var)

if is_collision_var == is_collision_var_quantum:
    print("the result of quantum algorithm is identical with classical algorithm")
else:
    print("the result of quantum algorithm is different from classical algorithm")
if farthest_connect != nearest_connect:
    print("is_collision_in_chunk_arr:")
    print(is_collision_in_chunk_arr)
    print("is_collision_in_chunk_arr_quantum:")
    print(is_collision_in_chunk_arr_quantum)

quantum result:
0
classical result:
1
the result of quantum algorithm is different from classical algorithm
is_collision_in_chunk_arr:
[1. 0. 1. 1. 1. 1. 0. 1.]
is_collision_in_chunk_arr_quantum:
[1. 0. 1. 1. 1. 1. 0. 1.]
